In [5]:
import pickle
from collections import defaultdict

In [2]:
import pandas as pd
df = pd.read_excel('../raw_data/Surgeon Mapping.xlsx')

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  


In [3]:
surgeon_mapping = defaultdict(set)
for _, row in df.iterrows():
    surgeon_mapping[row["CaseID"]].add(row['surgeonid'])
surgeon_mapping = {
    cid: list(sset)[0]
    for cid, sset in surgeon_mapping.items()
}

Calc stat

# Dup subseq 

In [8]:
with open("duplicated_seq_occlusion.pl", "rb") as f:
    occlusions = pickle.load(f)

In [9]:
pos_stat = defaultdict(int)
neg_stat = defaultdict(int)
surgeon_stat = defaultdict(lambda: defaultdict(int))

for i in occlusions:
    occl = occlusions[i]
    L, R, label, id = occl
    surgeon = surgeon_mapping[id]
    
    if label == 1:
        d = pos_stat
    else:
        d = neg_stat
    for LR in [L, R]:
        for starting, (seq, score) in LR.items():
            d[tuple(seq)] += 1
            
            surgeon_stat[surgeon][tuple(seq)] += 1

Pos sample top 10 dup seq

In [10]:
k = 10
top_k_common = sorted(pos_stat,
       key=lambda k: pos_stat[k],
       reverse=True)[:k]
print("\n".join([
    f"{seq} ({pos_stat[seq]})"
    for seq in top_k_common
]))

('p', 'p', 'p') (183)
('p', 'p', 'p', 'p') (107)
('c', 'c', 'c') (98)
('p', 'p', 'p', 'p', 'p') (56)
('c', 'c', 'c', 'c') (36)
('r', 'r', 'r') (36)
('p', 'p', 'p', 'p', 'p', 'p') (34)
('c', 'c', 'c', 'c', 'c') (27)
('p', 'p', 'p', 'p', 'p', 'p', 'p') (19)
('p', 'p', 'p', 'p', 'p', 'p', 'p', 'p') (16)


Neg sample top 10 dup seq

In [82]:
k = 10
top_k_common = sorted(neg_stat,
       key=lambda k: neg_stat[k],
       reverse=True)[:k]
print("\n".join([
    f"{seq} ({neg_stat[seq]})"
    for seq in top_k_common
]))

('p', 'p', 'p') (271)
('c', 'c', 'c') (153)
('p', 'p', 'p', 'p') (133)
('p', 'p', 'p', 'p', 'p') (84)
('c', 'c', 'c', 'c') (79)
('p', 'p', 'p', 'p', 'p', 'p') (55)
('c', 'c', 'c', 'c', 'c') (47)
('r', 'r', 'r') (38)
('p', 'p', 'p', 'p', 'p', 'p', 'p') (35)
('s', 's', 's') (23)


top 10 for each surgeon

In [154]:
k = 10

for sid, d in surgeon_stat.items():
    print("+"*20)
    print(f"surgeon {sid}")
    top_k_common = sorted(d,
       key=lambda k: d[k],
       reverse=True)[:k]
    print("\t"+"\n\t".join([
        f"{seq} ({d[seq]})"
        for seq in top_k_common
    ]))
    print()

++++++++++++++++++++
surgeon 26
	('c', 'c', 'c') (8)
	('p', 'p', 'p') (6)
	('c', 'c', 'c', 'c', 'c') (4)
	('p', 'p', 'p', 'p') (3)
	('c', 'c', 'c', 'c') (3)
	('p', 'p', 'p', 'p', 'p', 'p') (2)
	('p', 'p', 'p', 'p', 'p', 'p', 'p', 'p') (1)
	('c', 'c', 'c', 'c', 'c', 'c', 'c') (1)
	('r', 'r', 'r') (1)
	('p', 'p', 'p', 'p', 'p', 'p', 'p') (1)

++++++++++++++++++++
surgeon 17
	('p', 'p', 'p') (106)
	('p', 'p', 'p', 'p') (69)
	('c', 'c', 'c') (47)
	('p', 'p', 'p', 'p', 'p') (27)
	('p', 'p', 'p', 'p', 'p', 'p') (26)
	('c', 'c', 'c', 'c') (18)
	('s', 's', 's') (13)
	('p', 'p', 'p', 'p', 'p', 'p', 'p', 'p') (13)
	('r', 'r', 'r') (12)
	('s', 's', 's', 's', 's') (12)

++++++++++++++++++++
surgeon 7
	('p', 'p', 'p') (83)
	('c', 'c', 'c') (70)
	('c', 'c', 'c', 'c') (40)
	('c', 'c', 'c', 'c', 'c') (32)
	('p', 'p', 'p', 'p') (31)
	('p', 'p', 'p', 'p', 'p') (20)
	('r', 'r', 'r') (19)
	('s', 's', 's') (14)
	('c', 'c', 'c', 'c', 'c', 'c') (14)
	('c', 'c', 'c', 'c', 'c', 'c', 'c') (12)

++++++++++++++++

# Subseq occlusion

Even among all subseq, the recurring pattern is still salient

In [20]:
def calc_stat(occlusions):
    pos_stat = []
    neg_stat = []
    surgeon_stat = defaultdict(list)

    for i in occlusions:
        occl = occlusions[i]
        L, R, label, id = occl
        surgeon = surgeon_mapping[id]

        if label == 1:
            d = pos_stat
        else:
            d = neg_stat
        for LR in [L, R]:
            for starting, seq, score in LR:
                tup = (seq, score, starting, id, label)
                d.append(tup)
                surgeon_stat[surgeon].append(tup)
    return pos_stat, neg_stat, surgeon_stat

def show_pos_neg(stat, k=10):
    most_salient_seq = sorted(
        stat, key=lambda t: abs(t[1]),
        reverse=True
    )[:k]
    print("\n".join([
        f"{seq} ({score}) [from id={id}[label={label}], indices={starting}]"
        for seq, score, starting, id, label in most_salient_seq
    ]))
    
def show_surgeon(surgeon_stat, k=10):
    for surgeon, stat in surgeon_stat.items():
        print("+"*20)
        print(f"surgeon {surgeon}")

        most_salient_seq = sorted(
            stat, key=lambda t: abs(t[1]),
            reverse=True
        )[:k]

        print("\t"+"\n\t".join([
            f"{seq} ({score}) [from id={id}[label={label}], indices={starting}]"
            for seq, score, starting, id, label in most_salient_seq
        ]))
        print()

Window size = 10

In [21]:
window_size = 10
with open(f"subseq_occlusion-{window_size}.pl", "rb") as f:
    occlusions = pickle.load(f)
    
pos_stat, neg_stat, surgeon_stat = calc_stat(occlusions)

In [22]:
print("These are from cases where label = True")
show_pos_neg(pos_stat)

These are from cases where label = True
['p', 'g', 'm', 'r', 'c', 'c', 'p', 'c', 'p', 'm'] (0.006685888394713402) [from id=P-088[label=1], indices=[73 74 75 76 77 78 79 80 81 82]]
['c', 'p', 'p', 'c', 'p', 'p', 'p', 'p', 'p', 'm'] (0.006165320985019207) [from id=P-005[label=1], indices=[283 284 285 286 287 288 289 290 291 292]]
['c', 'p', 'g', 'm', 'r', 'c', 'c', 'p', 'c', 'p'] (0.006064181216061115) [from id=P-088[label=1], indices=[72 73 74 75 76 77 78 79 80 81]]
['p', 'p', 'p', 'p', 'r', 'p', 'p', 'p', 'm', 'm'] (0.005871623754501343) [from id=P-108[label=1], indices=[396 397 398 399 400 401 402 403 404 405]]
['r', 'p', 'p', 'p', 'p', 'p', 'p', 'm', 'r', 'r'] (0.005868242587894201) [from id=P-050[label=1], indices=[164 165 166 167 168 169 170 171 172 173]]
['p', 'p', 'p', 'c', 's', 's', 'p', 'p', 'p', 'm'] (0.0057752192951738834) [from id=P-002[label=1], indices=[283 284 285 286 287 288 289 290 291 292]]
['c', 'p', 'p', 'r', 'p', 'c', 'p', 'p', 'p', 'm'] (0.0057641323655843735) [fro

In [23]:
print("These are from cases where label = False")

show_pos_neg(neg_stat)

These are from cases where label = False
['c', 'p', 'p', 'p', 'p', 'p', 'c', 'c', 'p', 'm'] (0.007738795131444931) [from id=P-046[label=0], indices=[584 585 586 587 588 589 590 591 592 593]]
['c', 'c', 'p', 'p', 'p', 'p', 'p', 'c', 'c', 'p'] (0.006996050477027893) [from id=P-046[label=0], indices=[583 584 585 586 587 588 589 590 591 592]]
['r', 'p', 'r', 'r', 'p', 'p', 'p', 'p', 'p', 'm'] (0.0058815982192754745) [from id=P-091[label=0], indices=[100 101 102 103 104 105 106 107 108 109]]
['p', 'r', 'p', 'r', 'r', 'p', 'p', 'p', 'p', 'p'] (0.005622516386210918) [from id=P-091[label=0], indices=[ 99 100 101 102 103 104 105 106 107 108]]
['p', 'p', 'c', 'c', 'c', 'p', 'p', 'p', 'p', 'm'] (0.005591969937086105) [from id=P-122[label=0], indices=[287 288 289 290 291 292 293 294 295 296]]
['c', 'c', 'c', 'p', 'p', 'p', 'p', 'p', 'c', 'c'] (0.005468694027513266) [from id=P-046[label=0], indices=[582 583 584 585 586 587 588 589 590 591]]
['p', 'p', 'c', 'p', 'p', 'p', 'c', 'p', 'p', 'm'] (0.0053

In [24]:
show_surgeon(surgeon_stat)

++++++++++++++++++++
surgeon 26
	['c', 'c', 'p', 'p', 'p', 'm', 'm', 'r', 'r', 'r'] (0.0040044281631708145) [from id=P-156[label=1], indices=[202 203 204 205 206 207 208 209 210 211]]
	['c', 'c', 'c', 'p', 'p', 'p', 'm', 'm', 'r', 'r'] (0.003736968617886305) [from id=P-156[label=1], indices=[201 202 203 204 205 206 207 208 209 210]]
	['c', 'p', 'm', 'p', 'p', 'c', 'p', 'p', 'p', 'p'] (0.003081081435084343) [from id=P-156[label=1], indices=[205 206 207 208 209 210 211 212 213 214]]
	['p', 'c', 'c', 'c', 'p', 'p', 'p', 'm', 'm', 'r'] (0.002818075940012932) [from id=P-156[label=1], indices=[200 201 202 203 204 205 206 207 208 209]]
	['p', 'p', 'c', 'c', 'c', 'p', 'p', 'p', 'm', 'm'] (0.0019557569175958633) [from id=P-156[label=1], indices=[199 200 201 202 203 204 205 206 207 208]]
	['p', 'c', 'p', 'm', 'p', 'p', 'c', 'p', 'p', 'p'] (0.001878887414932251) [from id=P-156[label=1], indices=[204 205 206 207 208 209 210 211 212 213]]
	['c', 'p', 'p', 'c', 'c', 'c', 'p', 'p', 'p', 'm'] (0.00151

Window size = 5

In [25]:
window_size = 5
with open(f"subseq_occlusion-{window_size}.pl", "rb") as f:
    occlusions = pickle.load(f)
    
pos_stat, neg_stat, surgeon_stat = calc_stat(occlusions)

In [26]:
print("These are from cases where label = True")

show_pos_neg(pos_stat)

These are from cases where label = True
['c', 'p', 'c', 'p', 'm'] (0.006476186215877533) [from id=P-088[label=1], indices=[78 79 80 81 82]]
['c', 'c', 'p', 'c', 'p'] (0.006054031662642956) [from id=P-088[label=1], indices=[77 78 79 80 81]]
['s', 'p', 'p', 'p', 'm'] (0.005111689679324627) [from id=P-002[label=1], indices=[288 289 290 291 292]]
['p', 'p', 'p', 'm', 'm'] (0.00506133958697319) [from id=P-108[label=1], indices=[401 402 403 404 405]]
['p', 'p', 'p', 'p', 'm'] (0.004917674697935581) [from id=P-005[label=1], indices=[288 289 290 291 292]]
['p', 'p', 'p', 'p', 'p'] (0.004773784428834915) [from id=P-005[label=1], indices=[287 288 289 290 291]]
['c', 'p', 'p', 'p', 'm'] (0.004737057723104954) [from id=P-047[label=1], indices=[309 310 311 312 313]]
['p', 'p', 'p', 'm', 'r'] (0.0046752747148275375) [from id=P-002[label=1], indices=[289 290 291 292 293]]
['p', 'p', 'p', 'p', 'p'] (0.004592427983880043) [from id=P-368[label=1], indices=[262 263 264 265 266]]
['p', 'c', 'p', 'p', 'p']

In [27]:
print("These are from cases where label = False")

show_pos_neg(neg_stat)

These are from cases where label = False
['p', 'c', 'c', 'p', 'm'] (0.005988807417452335) [from id=P-046[label=0], indices=[589 590 591 592 593]]
['p', 'p', 'c', 'c', 'p'] (0.005937179084867239) [from id=P-046[label=0], indices=[588 589 590 591 592]]
['p', 'p', 'p', 'p', 'p'] (0.0055609531700611115) [from id=P-091[label=0], indices=[104 105 106 107 108]]
['p', 'p', 'p', 'p', 'm'] (0.005286769010126591) [from id=P-091[label=0], indices=[105 106 107 108 109]]
['p', 'p', 'p', 'p', 'm'] (0.005155524238944054) [from id=P-122[label=0], indices=[292 293 294 295 296]]
['p', 'p', 'p', 'c', 'c'] (0.0049484302289783955) [from id=P-046[label=0], indices=[587 588 589 590 591]]
['p', 'p', 'c', 'p', 'p'] (0.004653478041291237) [from id=P-037[label=0], indices=[324 325 326 327 328]]
['c', 'p', 'p', 'p', 'p'] (0.004601398482918739) [from id=P-122[label=0], indices=[291 292 293 294 295]]
['p', 'c', 'p', 'p', 'm'] (0.004333069548010826) [from id=P-307[label=0], indices=[335 336 337 338 339]]
['p', 'p', '

In [28]:
show_surgeon(surgeon_stat)

++++++++++++++++++++
surgeon 26
	['p', 'm', 'm', 'r', 'r'] (0.0029190885834395885) [from id=P-156[label=1], indices=[206 207 208 209 210]]
	['m', 'm', 'r', 'r', 'r'] (0.0028210021555423737) [from id=P-156[label=1], indices=[207 208 209 210 211]]
	['p', 'p', 'm', 'm', 'r'] (0.002484154887497425) [from id=P-156[label=1], indices=[205 206 207 208 209]]
	['c', 'p', 'p', 'p', 'p'] (0.002397993579506874) [from id=P-156[label=1], indices=[210 211 212 213 214]]
	['p', 'p', 'p', 'm', 'm'] (0.0018541282042860985) [from id=P-156[label=1], indices=[204 205 206 207 208]]
	['p', 'c', 'p', 'p', 'p'] (0.0015959981828927994) [from id=P-156[label=1], indices=[209 210 211 212 213]]
	['c', 'p', 'p', 'p', 'm'] (0.0014422396197915077) [from id=P-156[label=1], indices=[203 204 205 206 207]]
	['c', 'c', 'p', 'p', 'p'] (0.0012899506837129593) [from id=P-156[label=1], indices=[202 203 204 205 206]]
	['p', 'p', 'c', 'p', 'p'] (0.0007662475109100342) [from id=P-156[label=1], indices=[208 209 210 211 212]]
	['c', 

Window size = 3

In [29]:
window_size = 3
with open(f"subseq_occlusion-{window_size}.pl", "rb") as f:
    occlusions = pickle.load(f)
pos_stat, neg_stat, surgeon_stat = calc_stat(occlusions)

In [30]:
print("These are from cases where label = True")


show_pos_neg(pos_stat)

These are from cases where label = True
['p', 'c', 'p'] (0.005034656263887882) [from id=P-088[label=1], indices=[79 80 81]]
['c', 'p', 'm'] (0.004092208109796047) [from id=P-088[label=1], indices=[80 81 82]]
['p', 'p', 'p'] (0.004042724147439003) [from id=P-002[label=1], indices=[289 290 291]]
['c', 'p', 'c'] (0.0038438523188233376) [from id=P-088[label=1], indices=[78 79 80]]
['m', 'm', 'm'] (0.0037932172417640686) [from id=P-368[label=1], indices=[367 368 369]]
['p', 'p', 'p'] (0.003702659159898758) [from id=P-047[label=1], indices=[310 311 312]]
['p', 'p', 'm'] (0.00369179155677557) [from id=P-002[label=1], indices=[290 291 292]]
['p', 'p', 'p'] (0.0035015782341361046) [from id=P-005[label=1], indices=[289 290 291]]
['p', 'p', 'm'] (0.003399267792701721) [from id=P-103[label=1], indices=[297 298 299]]
['p', 'p', 'p'] (0.0032419986091554165) [from id=P-141[label=1], indices=[496 497 498]]


In [31]:
print("These are from cases where label = False")

show_pos_neg(neg_stat)

These are from cases where label = False
['c', 'c', 'p'] (0.004373950883746147) [from id=P-046[label=0], indices=[590 591 592]]
['g', 'g', 'g'] (-0.003993449732661247) [from id=P-090[label=0], indices=[57 58 59]]
['p', 'c', 'c'] (0.003930579870939255) [from id=P-046[label=0], indices=[589 590 591]]
['p', 'p', 'p'] (0.0037832511588931084) [from id=P-091[label=0], indices=[106 107 108]]
['p', 'c', 'p'] (0.0037312302738428116) [from id=P-303[label=0], indices=[68 69 70]]
['c', 'p', 'm'] (0.003686007112264633) [from id=P-046[label=0], indices=[591 592 593]]
['p', 'p', 'p'] (0.00364865455776453) [from id=P-122[label=0], indices=[293 294 295]]
['c', 'p', 'p'] (0.0034197382628917694) [from id=P-369[label=0], indices=[105 106 107]]
['p', 'm', 'm'] (0.003337548114359379) [from id=P-042[label=0], indices=[550 551 552]]
['c', 'p', 'c'] (0.0032999031245708466) [from id=P-051[label=0], indices=[103 104 105]]


In [32]:
show_surgeon(surgeon_stat)

++++++++++++++++++++
surgeon 26
	['r', 'r', 'r'] (0.0022853445261716843) [from id=P-156[label=1], indices=[209 210 211]]
	['m', 'r', 'r'] (0.0021089566871523857) [from id=P-156[label=1], indices=[208 209 210]]
	['p', 'p', 'p'] (0.0019608205184340477) [from id=P-156[label=1], indices=[212 213 214]]
	['p', 'p', 'p'] (0.001487918198108673) [from id=P-156[label=1], indices=[211 212 213]]
	['m', 'm', 'r'] (0.0013924883678555489) [from id=P-156[label=1], indices=[207 208 209]]
	['p', 'p', 'p'] (0.0012485701590776443) [from id=P-156[label=1], indices=[204 205 206]]
	['p', 'm', 'm'] (0.0012043584138154984) [from id=P-156[label=1], indices=[206 207 208]]
	['p', 'p', 'm'] (0.0012017805129289627) [from id=P-156[label=1], indices=[205 206 207]]
	['c', 'p', 'p'] (0.0006659766659140587) [from id=P-156[label=1], indices=[203 204 205]]
	['c', 'p', 'p'] (0.0005137231200933456) [from id=P-156[label=1], indices=[210 211 212]]

++++++++++++++++++++
surgeon 17
	['p', 'p', 'p'] (0.004042724147439003) [from 